### A1.2.6. Dependency Injection Pattern

> *A technique whereby one object supplies the dependencies of another object, rather than the dependent object creating them itself.*
>
> — Martin Fowler

**Explanation:**

**Dependency Injection (DI)** is a pattern where a class receives its dependencies from the outside rather than constructing them internally. The three common injection mechanisms are:

1. **Constructor injection** — dependencies are passed through the constructor.
2. **Setter injection** — dependencies are set via a method after construction.
3. **Interface injection** — the dependency provides an injector method that clients call.

Constructor injection is the most common and preferred form: it makes dependencies explicit and enforces that the object is fully initialized at construction time.

DI is the *mechanism* that realizes the Dependency Inversion Principle. Where DIP is the design guideline (depend on abstractions), DI is the concrete technique (pass abstractions in from outside).

**Example:**

An `OrderService` needs a `PaymentGateway` and a `InventoryChecker`. Instead of creating these internally, it receives them through its constructor. In tests, mock implementations can be injected. In production, real implementations are injected.

In [ ]:
from abc import ABC, abstractmethod


class PaymentGateway(ABC):
    @abstractmethod
    def charge(self, amount):
        pass


class InventoryChecker(ABC):
    @abstractmethod
    def is_available(self, item_id):
        pass


class StripeGateway(PaymentGateway):
    def charge(self, amount):
        return f"Charged ${amount:.2f} via Stripe"


class WarehouseInventory(InventoryChecker):
    def __init__(self, stock):
        self.stock = stock

    def is_available(self, item_id):
        return item_id in self.stock


class OrderService:
    def __init__(self, payment_gateway, inventory_checker):
        self.payment_gateway = payment_gateway
        self.inventory_checker = inventory_checker

    def place_order(self, item_id, amount):
        available = self.inventory_checker.is_available(item_id)
        if not available:
            return f"Item {item_id} is out of stock"
        return self.payment_gateway.charge(amount)


gateway = StripeGateway()
inventory = WarehouseInventory({"BOOK-001", "PEN-042"})
service = OrderService(gateway, inventory)

print(service.place_order("BOOK-001", 29.99))
print(service.place_order("MUG-999", 12.00))

**Python Library — `dependency-injector`:**

The [`dependency-injector`](https://pypi.org/project/dependency-injector/) library automates the wiring of dependencies in large applications. It provides a `Container` that declares how dependencies are constructed and resolved, eliminating manual constructor wiring.

```python
from dependency_injector import containers, providers

class Container(containers.DeclarativeContainer):
    gateway = providers.Singleton(StripeGateway)
    inventory = providers.Singleton(WarehouseInventory, stock={"BOOK-001", "PEN-042"})
    order_service = providers.Factory(OrderService, payment_gateway=gateway, inventory_checker=inventory)

container = Container()
service = container.order_service()
print(service.place_order("BOOK-001", 29.99))
```

The library handles singleton/factory scoping, configuration binding, and overriding for tests — replacing the manual constructor injection shown above with declarative container configuration.

**References:**

[📘 Fowler, M. (2004). *Inversion of Control Containers and the Dependency Injection Pattern.* martinfowler.com.](https://martinfowler.com/articles/injection.html)

[📘 Gamma, E., Helm, R., Johnson, R. & Vlissides, J. (1994). *Design Patterns: Elements of Reusable Object-Oriented Software.* Addison-Wesley.](https://www.pearson.com/en-us/subject-catalog/p/design-patterns-elements-of-reusable-object-oriented-software/P200000009480)

---

[⬅️ Previous: Adapter Pattern](./05_adapter_pattern.ipynb)